In [150]:
import pandas as pd
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.common.action_chains import ActionChains
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.common.by import By
from selenium.webdriver.support import expected_conditions as EC
from selenium.common.exceptions import NoSuchElementException
from selenium import webdriver
from selenium.webdriver.chrome.options import Options
from webdriver_manager.chrome import ChromeDriverManager
from bs4 import BeautifulSoup
from datetime import datetime
import time
import re
import logging
import traceback
import numpy as np
from random import randint
import sys
import itertools
from threading import Thread
from time import sleep

In [79]:
driver = webdriver.Chrome()
wait = WebDriverWait(driver, 8)

In [80]:
x = driver.get("https://www.google.com/maps/place/Moderna+Museet/@59.3262534,18.084163,17z/data=!4m7!3m6!1s0x465f9d566882083f:0xb219a21a1ce7e697!8m2!3d59.3259716!4d18.0844999!9m1!1b1")
time.sleep(4)

In [81]:
def clearwidgets(x):
                menu_bt = wait.until(EC.element_to_be_clickable((By.XPATH, '//*[@id="yDmH0d"]/c-wiz/div/div/div/div[2]/div[1]/div[4]/div[1]/div/button')))  
                menu_bt.click()

                n_menu_bt = wait.until(EC.element_to_be_clickable((By.XPATH, '//*[@id="yDmH0d"]/c-wiz/div/div/div/div[2]/div[3]/div[2]/div/div[2]/div[1]/div/button')))
                m_menu_bt = wait.until(EC.element_to_be_clickable((By.XPATH, '//*[@id="yDmH0d"]/c-wiz/div/div/div/div[2]/div[4]/div[2]/div/div[2]/div[1]/div/button')))
                o_menu_bt = wait.until(EC.element_to_be_clickable((By.XPATH, '//*[@id="yDmH0d"]/c-wiz/div/div/div/div[2]/div[5]/div[2]/div[2]/div/div[2]/div[1]/div/button')))

                n_menu_bt.click()
                m_menu_bt.click()
                o_menu_bt.click()

                widget = wait.until(EC.element_to_be_clickable((By.XPATH, '//*[@id="yDmH0d"]/c-wiz/div/div/div/div[2]/form/div/button')))  
                widget.click()

In [82]:
def sorting(x):
            drop_menu = wait.until(EC.element_to_be_clickable((By.XPATH, '//*[@id="pane"]/div/div[1]/div/div/div[2]/div[8]/button')))
            drop_menu.click()
            time.sleep(2)
            latest = wait.until(EC.element_to_be_clickable((By.XPATH, '//*[@id="action-menu"]/ul/li[2]')))
            latest.click()
            time.sleep(2) 

In [159]:
def scrolling(x):
            #Hitta totala antalet recensioner
            total_number_of_reviews = driver.find_element(By.XPATH,'//*[@id="pane"]/div/div[1]/div/div/div[2]/div[2]/div/div[2]/div[2]').text.split(" ")[0:2]
            total_number_of_reviews = ''.join(total_number_of_reviews)
            total_number_of_reviews = int(total_number_of_reviews)
            #Hitta diven för scrollning
            scrollable_div = driver.find_element(By.XPATH,'//*[@id="pane"]/div/div[1]/div/div/div[2]') #Kanske om vi går via klassen istället?
            #Skapa en kill-switch/timer
            sixMinutes = Thread(target=lambda:sleep(400)) # 6 min
            sixMinutes.start()
            for i in range(0,(round(total_number_of_reviews/10 - 1))): #Här har vi t.ex. 145 iterationer för Moderna Museet (1303/9)
                    driver.execute_script('arguments[0].scrollTop = arguments[0].scrollHeight', scrollable_div)
                    time.sleep(randint(2,7)*0.7)
                    if not sixMinutes.is_alive(): break

In [ ]:
def applybs4(x):
            response = BeautifulSoup(driver.page_source, 'html.parser')
            reviews = response.find_all('div', class_='ODSEW-ShBeI NIyLF-haAclf gm2-body-2')
            rev_dict = {'Review Rate': [],
                'Review Time': [],
                'Review Text' : []}
            for review in reviews:
                review_rate = review.find('span', class_='ODSEW-ShBeI-H1e3jb')["aria-label"]
                review_time = review.find('span',class_='ODSEW-ShBeI-RgZmSc-date').text
                review_text = review.find('span',class_='ODSEW-ShBeI-text').text
                rev_dict['Review Rate'].append(review_rate)
                rev_dict['Review Time'].append(review_time)
                rev_dict['Review Text'].append(review_text)  
            return(pd.DataFrame(rev_dict))

In [85]:
clearwidgets(x)

In [87]:
sorting(x)

In [90]:
scrolling(x)

In [92]:
df = applybs4(x)

In [93]:
df #Moderna Muséet

,Review Rate,Review Time,Review Text
0,5 stjärnor,för 2 dagar sedan,
1,5 stjärnor,för 3 dagar sedan,
2,5 stjärnor,för 5 dagar sedan,personalen svarar på frågor. lider av ett synd...
3,5 stjärnor,för 5 dagar sedan,Me encanto este museo.\n\nSi queres conocer má...
4,5 stjärnor,för 5 dagar sedan,
...,...,...,...
925,5 stjärnor,för 4 år sedan,
926,4 stjärnor,för 4 år sedan,(Översatt av Google) Vackert och fantastiskt a...
927,5 stjärnor,för 4 år sedan,(Översatt av Google) så trevligt\n\n(Original)...
928,4 stjärnor,för 4 år sedan,


In [131]:
driver.close()

In [136]:
driver = webdriver.Chrome()
wait = WebDriverWait(driver, 8)
x = driver.get("https://www.google.se/maps/place/Astrid+Lindgrens+V%C3%A4rld/@57.6742381,15.8404989,17z/data=!4m7!3m6!1s0x4659c95d7ddcd843:0xb084684f2c4e701f!8m2!3d57.6742381!4d15.8426876!9m1!1b1")

In [137]:
clearwidgets(x)

In [139]:
sorting(x)

In [155]:
scrolling(x)

In [156]:
df2 = applybs4(x)

In [157]:
df2 #Astrid Lindgrens Värld

,Review Rate,Review Time,Review Text
0,3 stjärnor,för 4 dagar sedan,Mycket folk och långa köer inne där jag tyckte...
1,5 stjärnor,för en vecka sedan,
2,5 stjärnor,för en månad sedan,(Översatt av Google) Anledningen till att vi v...
3,5 stjärnor,för 2 månader sedan,
4,5 stjärnor,för 2 månader sedan,
...,...,...,...
925,5 stjärnor,för 7 månader sedan,"(Översatt av Google) Snyggt presenterade, teat..."
926,5 stjärnor,för 7 månader sedan,Mysig park. Ska besökas igen synd bara med all...
927,3 stjärnor,för 7 månader sedan,"(Översatt av Google) Okej för barn, tråkigt fö..."
928,5 stjärnor,för 7 månader sedan,
